# FlockMTL Extension Tutorial

### You work for an e-commerce company that sells consumer electronics. You have a treasure trove of customer reviews, and your goal is to:

#### 1. Analyze the sentiment of each review.
#### 2. Identify high-impact reviews that require urgent attention.
#### 3. Extract recurring themes or topics to understand common issues and areas for product improvement.
#### 4. Track customer satisfaction trends over time.

### Let us see how to build SQL queries that use `FlockMTL` to achieve all of this.


#### Step 1 : Create your OPENAI_API env file

Create a file called .env in the current working directory, with the following text:

`export OPENAI_API_KEY='<your OPENAI API KEY'`

Replace the placeholder with your API key.

#### Step 2 : Load your OPENAI_API env file

We use the dotenv python package

In [112]:
!pip install python-dotenv
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

True

#### Step 3: Install and import DuckDB

We are using DuckDB version 1.1.1 for this tutorial

In [113]:
#!pip install duckdb==1.1.1
import duckdb
print(duckdb.__version__)


1.1.1


If everything goes well, you should see the version being printed, its 1.1.1 for our example

#### Step 4: Create a database for our use

1. We create a new persistent database connection for our example called `mydb.db`.\
2. We also pass a config dict `config={'allow_unsigned_extensions' : 'true'}` to allow us to load the extension.
3. We also load a CSV file called `product_reviews.csv` into a table called `product_reviews`

In [115]:
# Create a connection to an in-memory database
con = duckdb.connect('mydb.db', config={'allow_unsigned_extensions' : 'true'})

csv_path = 'product_reviews.csv' #add your own path
con.execute(f"CREATE TABLE product_reviews AS SELECT * FROM read_csv_auto('{csv_path}')");


#### Step 5: Load our extension


We use the DuckDB `INSTALL` and `LOAD` commands to load our extension\
Set the `path` variable to the location where the extension binary is downloaded, and then execute the following


In [116]:
path = '/home/sunny/work/large_flock_extension/large_flock.duckdb_extension' # replace your path here
con.execute(f"INSTALL '{path}'")
con.execute("LOAD large_flock")

### And that's it ! Our extension is loaded, now we can use the awesome LLM capabilities inside for some fun task!
You can go through the available prompts and models we have by executing the following:


In [117]:
available_prompts = con.execute("get prompt;").fetchall()
available_models = con.execute("get model;").fetchall()

print(available_prompts)
print(available_models)

[('summarize', 'Summarize the following text: {{text}}'), ('validate_email', 'validate this email: {{email}}'), ('test', 'is this email valid {{email}} the output should be a bool value'), ('check john', 'return the values with the first_name John'), ('keywords', 'extract the keywords from the next text'), ('is_high_impact_review_prompt', 'Determine if the given review is a high-impact review that provides valuable insights. Consider the following factors:\n\n1. Sentiment: {sentiment}\n2. Star Rating: {rating}\n3. Review Length: {review_length}\n\nA high-impact review typically has:\n\n- A strong sentiment (very positive or very negative)\n- An extreme rating (1-2 or 4-5 stars)\n- Sufficient length to provide detailed feedback (usually more than 50 words)\n\nOutput your decision as a boolean (true for high-impact, false for low-impact) with a brief explanation.'), ('sentiment_analysis_prompt', 'Analyze the sentiment of the following product review. Consider both the review text and the

#### Let's go through the tasks one by one

#### Performing Sentiment Analysis on Product Reviews:
The first step is to analyze the sentiment of each review. We can use LLMs to generate a detailed sentiment analysis by combining the review text with the star rating.

We use `lf_map`, a function that calls the `semantic analyzer` model to perform sentiment analysis on each review.\
`semantic analyzer` is an alias for `gpt-4o-mini` \
The model evaluates the sentiment based on both the review text and the star rating.
This step gives us a sentiment score or label (e.g., positive, negative, neutral) for each review, providing a solid foundation for deeper analysis.


In [121]:
query = """
CREATE TABLE sentiment_analysis AS
WITH sentiment_analysis AS (
    SELECT 
        ProductID as product_id, 
        ID AS review_id,
        Review AS review_text, 
        Rating AS star_rating, 
        lf_map('sentiment_analysis_prompt', 'semantic_analyzer', {'review': review_text, 'rating': star_rating}) AS sentiment_json
    FROM 
        product_reviews
)
SELECT * 
FROM sentiment_analysis;
"""

# Run the query and fetch results
results = con.execute(query).fetchall()

for row in results:
    print(row)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(10,)


#### Lets take a look at the newly created sentiment_analysis table

In [122]:
query = "select * from sentiment_analysis;"
results = con.execute(query).fetchall()

for row in results:
    print(row)

(101, 1, 'This smartphone is amazing! The camera quality is outstanding, and the battery life is impressive. Highly recommended!', 5, '{"explanation":"The review expresses strong enthusiasm and satisfaction with the smartphone, highlighting its camera quality and battery life positively.","rating":"5","review":"This smartphone is amazing! The camera quality is outstanding, and the battery life is impressive. Highly recommended!","sentiment":"positive"}')
(101, 2, "Decent phone, but the battery drains too quickly. Otherwise, it's okay.", 3, '{"explanation":"The review is mixed, acknowledging some decent features while also pointing out a significant drawback, leading to a neutral sentiment overall.","rating":"3","review":"Decent phone, but the battery drains too quickly. Otherwise, it\'s okay.","sentiment":"neutral"}')
(102, 3, 'These headphones are a game-changer. The noise cancellation is top-notch, and the sound quality is superb.', 5, '{"explanation":"The review conveys strong posit

#### Filtering for High-Impact Reviews
Not all reviews are equally important. Some are more detailed, more critical, or offer particularly valuable feedback. In this next step, we filter out the "high-impact" reviews—those that deserve immediate attention.

This query introduces lf_filter, which uses the current model to determine whether a review is "high-impact" based on the sentiment, rating, and length of the review. High-impact reviews could include:

Negative reviews that are detailed and provide insights into product issues.
Positive reviews that highlight key features or advantages (useful for marketing).
By focusing on these high-impact reviews, you can address critical feedback and leverage valuable testimonials effectively.


In [123]:
query = """
CREATE TABLE filtered_reviews AS
WITH filtered_reviews AS (
    SELECT 
        * 
    FROM 
        sentiment_analysis 
    WHERE 
        lf_filter('is_high_impact_review_prompt', 'gpt-4o', {
            'sentiment': sentiment_json, 
            'rating': star_rating, 
            'review_length': LENGTH(review_text)
        })
)
SELECT * 
FROM filtered_reviews;
"""

# Run the query and fetch results
results = con.execute(query).fetchall()

for row in results:
    print(row)

(7,)


#### Extracting Key Themes from Reviews

Once you’ve identified high-impact reviews, the next step is understanding why customers leave them.
This query helps identify recurring themes such as complaints about battery life, praises for design, or issues with customer service.
This query extracts common themes using LLM's ability to understand and categorize text. You’ll get insights like:

Common Complaints: "The battery drains too quickly."
Positive Feedback: "The screen resolution is excellent."

We first define our own prompt called `extract_themes`, and then use it to extract the themes from `filtered_reviews`

In [139]:
# Define the extract themes prompt
extract_themes_prompt = """
Analyze the following sentiment analysis JSON. Identify and extract key themes or topics discussed in the product review. Output the themes in a JSON array format.

Sentiment Analysis JSON: {sentiment_json}

Output your response in the following JSON format:
{
    "themes": ["theme1", "theme2", "theme3"]
}
"""

# Escape single quotes in the prompt for SQL
escaped_prompt = extract_themes_prompt.replace("'", "''")

# Create the SQL query to register the prompt
query = f"""
CREATE PROMPT ('extract_themes_prompt', '{escaped_prompt}');
"""

# Execute the query to create the prompt
results = con.execute(query).fetchall()


ConstraintException: Constraint Error: Duplicate key "prompt_name: extract_themes_prompt" violates primary key constraint. If this is an unexpected constraint violation please double check with the known index limitations section in our documentation (https://duckdb.org/docs/sql/indexes).

In [136]:
query = """
CREATE TABLE themes_extracted AS
WITH themes_extracted AS (
    SELECT 
        product_id, 
        review_id, 
        review_text,
        star_rating,
        lf_map('extract_themes_prompt', 'gpt-4o', {'sentiment_json': sentiment_json}) AS themes
    FROM 
        filtered_reviews
)
SELECT * 
FROM themes_extracted;
"""
# Run the query and fetch results
results = con.execute(query).fetchall()

for row in results:
    print(row)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(7,)


#### Tracking Customer Sentiment Over Time
Customer feedback changes as new product updates are rolled out or customer expectations evolve. To track customer satisfaction over time, you can analyze how the average sentiment and star ratings change month by month.

We use 2 different prompts here `sentiment_analysis_prompt` and `extract_sentiment_score_prompt_numeric`

In [137]:
# Define the extract sentiment score prompt
extract_sentiment_score_prompt_numeric = """
Analyze the following sentiment analysis JSON and extract the sentiment score. 
Output the sentiment score as a numeric value.

Sentiment Analysis JSON: {sentiment_json}

Output your response as a number {sentiment_score}
"""

# Define the SQL query to create the prompt
query = f"""
CREATE PROMPT ('extract_sentiment_score_prompt_numeric', '{extract_sentiment_score_prompt_numeric}');
"""

# Execute the query to create the prompt
con.execute(query)


In [138]:
# Define the SQL query
query = """
WITH sentiment_analysis_dated AS (
    SELECT 
        ProductID as product_id, 
        ID AS review_id,
        Review AS review_text, 
        Rating AS star_rating, 
        Date as review_date,
        lf_map('sentiment_analysis_prompt', 'gpt-4o', {'review': review_text, 'rating': star_rating}) AS sentiment_json
    FROM 
        product_reviews
)
SELECT 
    product_id, 
    EXTRACT(YEAR FROM review_date) AS year, 
    EXTRACT(MONTH FROM review_date) AS month, 
    AVG(star_rating) AS avg_star_rating, 
    AVG(
        CAST(
            (lf_map('extract_sentiment_score_prompt', 'gpt-4o', {'sentiment_json': sentiment_json}))->>'sentiment_score' AS DOUBLE
        )
    )
     AS avg_sentiment_score
FROM 
    sentiment_analysis_dated
GROUP BY 
    product_id, 
    EXTRACT(YEAR FROM review_date), 
    EXTRACT(MONTH FROM review_date)
ORDER BY 
    year DESC, month DESC;
"""

# Execute the query and fetch the results
results = con.execute(query).fetchall()

# Print the results
for row in results:
    print(row)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(101, 2024, 3, 4.0, 4.0)
(102, 2024, 3, 4.0, 4.0)
(103, 2024, 3, 2.5, 2.5)
(104, 2024, 3, 4.0, 4.0)
(105, 2024, 3, 2.5, 2.5)


### And that is it, we have successfully created new prompts, used existing prompts and manipulated the standard table to gain analytical insights in our data, using the power of LLMs!

We finally close the DuckDB connection. The data is persisted into out database `mydb.db`

In [110]:
# Close the DuckDB connection
con.close()


In [119]:
con.execute ("DROP TABLE filtered_reviews;")

In [120]:
con.execute ("DROP TABLE sentiment_analysis ;")

In [134]:
con.execute ("delete prompt 'extract_sentiment_score_prompt_numeric';")

In [140]:
con.execute ("delete prompt 'extract_themes_prompt';")

In [132]:
con.execute ("DROP TABLE themes_extracted ;")